In [15]:
import csv

train_commands = []

base_path = "/visinf/home/vilab25/Confidence-based-generative-data-Augmentation-for-Meta-Learning"
for mode in ["run_within_domain", "run_cross_domain"]:
    if "run_within_domain":
        mode_shortcut = "WD"
    else:
        mode_shortcut = "CD"
    

    for model in ["finetuning", "maml"]:
        
        for dropout_mode in ["baseline", "dropout"]:
            
            for k in [1, 5, 10]:
                cmd_strig = f"""{base_path}/scripts/isolated_run.sh \
                {base_path}/scripts/train/train.sh \
                --datasets_dir "/fastdata/vilab24/meta-album" \
                --{mode} \
                --model_dir "baselines/{model}" \
                --config_path "{base_path}/configs/train/{dropout_mode}/{mode_shortcut}/{dropout_mode}_k_{k}.yml" \
                --output_dir "/fastdata/vilab25/output/scheduler/training" \
                --log_file /visinf/home/vilab25/logs/train_{mode}_{model}_{dropout_mode}_k_{k}.log
                """
                
                train_commands.append(cmd_strig)
                
                

# Write commands to CSV file
csv_file_path = "scripts1.csv"
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["ID", "command", "status", "attached to process", "attached to gpu", "process alive", "expected_cpu_usage (in %)", "expected_gpu_usage (in MB)", "timestamp_started", "last_timestamp"])
    for index, cmd in enumerate(train_commands[::2], start=1):
        writer.writerow([index, cmd, "waiting", " ", " ", " ", "1800", "3500", " ", " "])

print(f"CSV file '{csv_file_path}' has been created successfully.")

# Write commands to CSV file
csv_file_path = "scripts2.csv"
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["ID", "command", "status", "attached to process", "attached to gpu", "process alive", "expected_cpu_usage (in %)", "expected_gpu_usage (in MB)", "timestamp_started", "last_timestamp"])
    for index, cmd in enumerate(train_commands[1::2], start=1):
        writer.writerow([index, cmd, "waiting", " ", " ", " ", "1800", "3500", " ", " "])

print(f"CSV file '{csv_file_path}' has been created successfully.")

CSV file 'scripts1.csv' has been created successfully.
CSV file 'scripts2.csv' has been created successfully.


In [1]:
def cmd_script_eval(base_path, k, mode, model_type):
    return f"""{base_path}/scripts/isolated_run.sh \
            {base_path}/scripts/train/train.sh \
            --datasets_dir "/fastdata/vilab24/meta-album" \
            --ks "{k}" \
            --{mode} \
            --model_path "/fastdata/vilab25/output/scheduler/training/{model_type}" \
            --model {model} \
            --config_path "configs/eval/standard_augmentation/mc_dropout/standard_augment_mc" \
            --output_dir "/fastdata/vilab25/output/scheduler/eval/" \
            --log_file /visinf/home/vilab25/logs/eval_{k}_{model}_{mode}.log
            """
    
def yaml_script_eval(k, augmentation_method, confidence_method):
    confidence_estimator_str = {"baseline": '"ConstantConfidenceProvider"',
                                "mc_dropout": '"MCDropoutConfidenceEstimator"',
                                "gt_confidence": '"GTConfidenceEstimator"'}[confidence_method]
        
    augmentation_method_str = {"baseline": 'null',
                                "generative_augmentation": '"GenerativeAugmentation"',
                                "pseudo_augmentation": '"PseudoAugmentation"',
                                "standard_augmentation": '"StandardAugmentation"'}[augmentation_method]
    
    
    return f"""dataset: 
test: 
    n_ways:
        value: 5
    k_shots: 
        value: {k + 10*(augmentation_method_str == '"PseudoAugmentation"')}
    query_size: 20
        
evaluation:
    tasks_per_dataset: 100
    confidence_estimators:
        use: {confidence_estimator_str}
        ConstantConfidenceProvider:
            confidence: {int(augmentation_method_str == 'null')} #1 if augmentation_method = baseline, if conidence_method = "baseline" = 0
	    GTConfidenceEstimator: {"{}"} 
        MCDropoutConfidenceEstimator:
            num_samples: 100

    augmentors:
	    use: {augmentation_method_str}
       	PseudoAugmentation: 
            keep_original_data: True
            augmentation_size: 
                scale: 1
                offset: 3 
                maximum: 20
                threshold: 0.8
        StandardAugmentation:
            threshold: 0.75
            scale: 2
            keep_original_data: True
            rotation: 25
            flip: True
        GenerativeAugmentation:
            keep_original_data: True
            augmentation_size: 
                scale: 1
                offset: 3 
                maximum: 20
                threshold: 0.8
            annotator_type: "hed"
"""

In [8]:
eval_commands = []
yaml_scripts = []
base_path = "/visinf/home/vilab25/Confidence-based-generative-data-Augmentation-for-Meta-Learning"

for k in [1, 5, 10]:
    for model in ["finetuning", "maml"]:
        for mode in ["run_within_domain", "run_cross_domain"]:
            for model_type in ["baseline", "dropout"]:
            
                if model_type == "baseline":
                    confidence_methods = ["baseline"]
                          
                elif model_type == "dropout":
                    confidence_methods = ["baseline", "mc_dropout"]

                for augmentation_method in ["baseline", "generative_augmentation", "standard_augmentation"]:
                    
                    if augmentation_method == "baseline":
                        confidence_methods = ["baseline"]
                                                                                       
                    
                    for confidence_method in confidence_methods:
                        eval_commands.append(cmd_script_eval(base_path, k, mode, model_type))
                        yaml_scripts.append(f"{k},{model},{mode},{model_type}"+yaml_script_eval(k, augmentation_method, confidence_method))
                                             
len(eval_commands)

72

In [28]:
import itertools

model_type = ["baseline", "dropout"]
confidence_method = ["baseline", "mc", "gt"]
data_aug = ["baseline", "standard_aug", "gen_aug", "pseudo_aug"]

ignored_config = []
configs = []
for config in itertools.product(model_type, confidence_method, data_aug):
    match config:
        case ("baseline", "baseline", "baseline"): 
            configs.append(config)
        case ("baseline", _, _): 
            ignored_config.append(config)
        case ("dropout", "baseline", _):
            configs.append(config)
        case ("dropout", _, "baseline"):
            ignored_config.append(config)
        case ("dropout", "mc" | "gt", _):
            configs.append(config)       
        case _:
            print(config)

print("Selected Config")
for config in configs:
    print(config)

print(len(configs))

print("Ignored Config")
for config in ignored_config:
    print(config)
    
print(len(ignored_config))


Selected Config
('baseline', 'baseline', 'baseline')
('dropout', 'baseline', 'baseline')
('dropout', 'baseline', 'standard_aug')
('dropout', 'baseline', 'gen_aug')
('dropout', 'baseline', 'pseudo_aug')
('dropout', 'mc', 'standard_aug')
('dropout', 'mc', 'gen_aug')
('dropout', 'mc', 'pseudo_aug')
('dropout', 'gt', 'standard_aug')
('dropout', 'gt', 'gen_aug')
('dropout', 'gt', 'pseudo_aug')
11
Ignored Config
('baseline', 'baseline', 'standard_aug')
('baseline', 'baseline', 'gen_aug')
('baseline', 'baseline', 'pseudo_aug')
('baseline', 'mc', 'baseline')
('baseline', 'mc', 'standard_aug')
('baseline', 'mc', 'gen_aug')
('baseline', 'mc', 'pseudo_aug')
('baseline', 'gt', 'baseline')
('baseline', 'gt', 'standard_aug')
('baseline', 'gt', 'gen_aug')
('baseline', 'gt', 'pseudo_aug')
('dropout', 'mc', 'baseline')
('dropout', 'gt', 'baseline')
13


In [23]:
len(base_product)

12

In [24]:
len(config_specific_product)

24

In [25]:
len(total_product)

288